# Data Cleaning with Pandas

In this notebook we'll go through a few basic data cleaning steps that should be performed on all new datasets where necessary.

We'll go through the process with both the `orders` and `orderlines` datasets. You can then practice these skills by cleaning the `products` dataset yourself

In [4]:
import pandas as pd

In [5]:
# orders.csv
url = "https://drive.google.com/file/d/1Vu0q91qZw6lqhIqbjoXYvYAQTmVHh6uZ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders = pd.read_csv(path)

# orderlines.csv
url = "https://drive.google.com/file/d/1FYhN_2AzTBFuWcfHaRuKcuCE6CWXsWtG/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines = pd.read_csv(path)



Before we begin, let's create a copy of the `orders` and `orderlines` DataFrames. This way we are sure any of our changes won't affect the original DataFrames

In [6]:
orders_df = orders.copy()

In [7]:
orderlines_df = orderlines.copy()

In [8]:
products_running_removed = 0
orderlines_running_removed = 0
orders_running_removed = 0

In [9]:
#products.csv
url = 'https://drive.google.com/file/d/1YM_FRc13lL0VvRJsZCqcQL30Aaxf-zr2/view?usp=drive_link'
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)



In [10]:
products_df = products.copy()

One of the best ways to begin data cleaning is by exploring using `.info()`. This will tell us:
* The shape of the DataFrame
* The names of the columns
* If there are any missing values
* The datatypes of the columns

By exploring the missing values and correcting any incorrect datatypes, we often come across inconsistencies in our data.

Beyond this, we should also have a **check for any duplicate rows**.

Let's first deal with the duplicates, as it's nice and easy, then we'll explore what `.info()` has to tell us.

## 1.&nbsp; Duplicates
We can check for duplicates using the pandas [.duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html) method.

We can then delete these rows, if we wish, using [.drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

In [11]:
# orders_df
orders_df.duplicated().sum()

np.int64(0)

In [12]:
# orderlines_df
orderlines_df.duplicated().sum()

np.int64(0)

In [13]:
# products_df
products_df.duplicated().sum()

np.int64(8746)

We have no duplicate rows in either DataFrame. Easy, there is no problem to solve. Normally though, if there were some duplicates, we'd drop the extra rows.

In [14]:
products_df.head(10)

,sku,name,desc,price,promo_price,in_stock,type
0,RAI0007,Silver Rain Design mStand Support,Aluminum support compatible with all MacBook,59.99,499.899,1,8696
1,APP0023,Apple Mac Keyboard Keypad Spanish,USB ultrathin keyboard Apple Mac Spanish.,59,589.996,0,13855401
2,APP0025,Mighty Mouse Apple Mouse for Mac,mouse Apple USB cable.,59,569.898,0,1387
3,APP0072,Apple Dock to USB Cable iPhone and iPod white,IPhone dock and USB Cable Apple iPod.,25,229.997,0,1230
4,KIN0007,Mac Memory Kingston 2GB 667MHz DDR2 SO-DIMM,2GB RAM Mac mini and iMac (2006/07) MacBook Pr...,34.99,31.99,1,1364
5,APP0073,Apple Composite AV Cable iPhone and iPod white,IPhone and iPod AV Cable Dock to Composite Video.,45,420.003,0,1230
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.471,0,1364
7,KIN0009,Mac Memory Kingston 2GB 800MHz DDR2 SO-DIMM,2GB RAM iMac with Intel Core 2 Duo (Penryn).,36.99,274.694,0,1364
8,KIN0001-2,Mac memory Kingston 4GB (2x2GB) 667MHz DDR2 SO...,RAM 4GB (2x2GB) Mac mini and iMac (2006/07) Ma...,74,669.904,0,1364
9,APP0100,Apple Adapter Mini Display Port to VGA,Adapter Mini Display Port to VGA MacBook and M...,35,330.003,0,1325


# 2.&nbsp; `.info()`

In [15]:
orders_df.info(), orders_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB


(None,
             order_id     total_paid
 count  226909.000000  226904.000000
 mean   413296.482480     569.225818
 std     65919.250331    1761.778002
 min    241319.000000       0.000000
 25%    356263.000000      34.190000
 50%    413040.000000     112.990000
 75%    470553.000000     525.980000
 max    527401.000000  214747.530000)

* `total_paid` has 5 missing values
* `created_date` should become datetime datatype

In [16]:
orderlines_df.info(), orderlines_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


(None,
                  id       id_order  product_id  product_quantity
 count  2.939830e+05  293983.000000    293983.0     293983.000000
 mean   1.397918e+06  419999.116544         0.0          1.121126
 std    1.530096e+05   66344.486479         0.0          3.396569
 min    1.119109e+06  241319.000000         0.0          1.000000
 25%    1.262542e+06  362258.500000         0.0          1.000000
 50%    1.406940e+06  425956.000000         0.0          1.000000
 75%    1.531322e+06  478657.000000         0.0          1.000000
 max    1.650203e+06  527401.000000         0.0        999.000000)

* `date` should be a datetime datatype
* `unit_price` should be a float datatype

## 3.&nbsp; Missing values

### 3.1.&nbsp; Orders
* `total_paid` has 5 missing values

In [17]:
num_missing = orders_df['total_paid'].isna().sum()

total_rows = orders_df.shape[0]
percent_missing = (100*num_missing/total_rows)
print(f"5 missing values represents {percent_missing:.5f}% of the rows in our DataFrame")
orders_df['total_paid'].isna().value_counts(normalize=True)

5 missing values represents 0.00220% of the rows in our DataFrame


,proportion
total_paid,
False,0.999978
True,0.000022


> A quick way to find out a percentage if you don't need to print out a sentence for yourself/students/colleagues is `.value_count(normalize=True)`

In [18]:
orders_df['total_paid'].isna().value_counts(normalize=True)

,proportion
total_paid,
False,0.999978
True,0.000022


As there is such a tiny amount of missing values, we will simply delete these rows, as we have enough data without them.

In [19]:
orders_df = orders_df.dropna(axis=0).copy()
orders_df

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled
...,...,...,...,...
226904,527397,2018-03-14 13:56:38,42.99,Place Order
226905,527398,2018-03-14 13:57:25,42.99,Shopping Basket
226906,527399,2018-03-14 13:57:34,141.58,Shopping Basket
226907,527400,2018-03-14 13:57:41,19.98,Shopping Basket


Should you have a significant number of missing values in the future, you have a choice:
+ you can impute the values
+ you can take the values from other DataFrames if they are redundantly stored
+ you can delete the rows or columns
+ or any number of other creative solutions

Please, always consider how much time you have on your project, and what impact your method of choice will have on your final assesment.

### 3.2.&nbsp; Orderlines
There are no missing values in `orderlines_df`

## 4.&nbsp; Datatypes

### 4.1.&nbsp; Orders
* `created_date` should become datetime datatype

In [20]:
orders_df["created_date"] = pd.to_datetime(orders_df["created_date"])
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 226904 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 8.7+ MB


### 4.1.&nbsp; Orderlines
* `date` should be a datetime datatype
* `unit_price` should be a float datatype

#### 4.1.1.&nbsp; `date`

In [21]:
orderlines_df["processing_date"] = pd.to_datetime(orderlines_df["processing_date"])

KeyError: 'processing_date'

In [ ]:
orderlines_df.info()

#### 4.1.2.&nbsp;`unit_price`

In [ ]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

As you can see when we try to convert `unit_price` to a numerical datatype, we receive a `ValueError` telling us that pandas doesn't understand the number `1.137.99`. This is probably because numbers cannot have multiple decimal points. Let's see if there are any other numbers like this:

> `.` is a wildcard in regex, we need the `\` as an escape

In [ ]:
# Count the number of decimal points in the unit_price
orderlines_df['unit_price'].str.count("\.").value_counts()

Looks like over 36000 rows in `orderlines` are affected by this problem. Let's work out how much that is as a percentage of our total data.

In [ ]:
products_running_removed = 0
orderlines_running_removed = 3600
orders_running_removed = 0

In [ ]:
# Count the rows with more than one `.`
mult_decimal_rows = (orderlines_df['unit_price'].str.count("\.")>1).sum()

# Find the percentage of corrupted rows
percent_corrupted = (100 * mult_decimal_rows / orderlines_df.shape[0])
print(f"{percent_corrupted:.2f}% of the rows in our DataFrame have multiple decimal points in the unit_price")

This is a bit of a tricky decision as 12.3% is a significant amount of our data... and we might even end up losing a larger portion of our data than this too. For the moment we will delete the rows as we only have 2 weeks for this project and I'd like some quick, accurate results to show. If we have time at the end, we can come back and investigate this problem further, maybe there's a solution?

Each row of `orderlines` represents a product in an order. For example, if order number 175 contained 3 seperate products, then order 175 would have 3 rows in `orderlines`, one row for each of the products. If 2 of those products have 'normal' prices (14.99, 15.85) and 1 has a price with 2 decimal points (1.137.99), we need to remove the whole order and not just the affected row. If we only remove the row with 2 decimal places then any later analysis about products and prices could be misleading.

We therefore need to find the order numbers associated with the rows that have 2 decimal points, and then remove all the associated rows.

In [ ]:
# Boolean mask to find the orders that contain a price with multiple decimal points
multiple_decimal_mask = orderlines_df['unit_price'].str.count(r"\.") > 1

# Apply the boolean mask to the orderlines DataFrame. This way we can find the order_id of all the affected orders.
corrupted_order_ids = orderlines_df.loc[multiple_decimal_mask, "id_order"]

# Keep only the rows that do not have multiple decimal points
orderlines_df = orderlines_df.loc[~orderlines_df['id_order'].isin(corrupted_order_ids)]

In [ ]:
orderlines_df.shape[0]

We still have 216250 rows in orderlines to work with. This should be more than enough for our evaluation.

Now that all of the 2 decimal point prices have been removed, let's try again to convert the column `unit_price` to the correct datatype.

In [ ]:
orderlines_df["unit_price"] = pd.to_numeric(orderlines_df["unit_price"])

In [ ]:
orderlines_df.describe() # there is a negative value
orderlines_df.sort_values(by="unit_price") # need check to see if there are other negative prices, only one and now we can compare that with the other prices associated with the sku

In [ ]:
orderlines_df.query("sku == 'APP1465'")["unit_price"].value_counts() # looks like 119 is not out of the range of the prices associate with product
orderlines_df['unit_price']= orderlines_df['unit_price'].abs()  #to make sure all values are positive # sku == APP1465


In [ ]:
#Many 999.99 values now I want to check that out
#orderlines_df.sort_values(by="unit_price")

high_amount_mask = (orderlines_df['unit_price'] == 999.99)
high_amount_mask.sum()

In [ ]:
(orderlines_df["unit_price"] == 999.99).sum() / orderlines_df.shape[0] # only 5% of the all the data I would leave it.

In [ ]:
orderlines_df.loc[high_amount_mask]

It worked perfectly

# Unwanted Columns and columns names.
In orderlines_df we see that product_id has no information for us to use. There is ID that could be set to an index if we wanted but that does not need to be done.

Also we see that id_order and order_id (orders) should probably match to make it easier to merge later in the analysis process.

In [ ]:
# dropping product_id and id from the order_lines



# new column named 'order_id' df.rename(columns={"A": "a", "B": "c"})

orderlines_df = orderlines_df.rename(columns={
    'id_order': 'order_id',
    'product_quantity': 'product_quantity',  # this one is actually unchanged, so you could remove it
    'sku': 'sku',                            # also unchanged
    'unit_price': 'unit_price',              # unchanged too
    'date': 'processing_date'
})

orderlines_df

orderlines_df = orderlines_df.drop(["id", "product_id"], axis=1).copy()



# Writing to file
After all this work, new CSV files can be written to persist the changes. This way, they need not be repeated before starting the analysis.

We can check "where we are" in our file system to help orient ourselves for choosing where to write the files. If you're in Colab, don't bother. Just write the name of the file and check the file browser on the left of the window. Be sure to download your CSV and save it somewhere (like your Drive) so you can use it later.

In [ ]:
orderlines_df.info()

In [ ]:
from google.colab import files

orders_qu.to_csv("orders_qu.csv", index=False)
files.download("orders_qu.csv")

orderlines_qu.to_csv("orderlines_qu.csv", index=False)
files.download("orderlines_qu.csv")

In [ ]:
# the indexes have no useful information, and so won't be written to file
orders_df.to_csv('orders_clean.csv',index=False)
orderlines_df.to_csv("orderlines_clean.csv", index=False)


In [ ]:
orders_df.info()

# Challenge: Clean the `products` DataFrame
Now it's your turn. Use the lessons you learnt above and clean the products DataFrame. You don't have to copy exactly what we did. Think about the consequences of your actions, sometimes it is ok to delete rows, other times you may wish to come up with more creative solutions.

In [ ]:

products_df = products.copy()

In [ ]:
# products.csv
url = "https://drive.google.com/file/d/1afxwDXfl-7cQ_qLwyDitfcCx3u7WMvkU/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products = pd.read_csv(path)

In [ ]:
# your code here
products_df.head(3)

### Look for Duplicates

In [ ]:
# your code here

products_df.duplicated(subset = "sku").sum()
# Filtered by the Unique ID number to make sure that there were truly multiple product names

In [ ]:
products.shape

In [ ]:
num_duplicats = products_df.duplicated().sum()
total_rows1 = products_df.shape[0]
percent_missing = (100*num_duplicats/total_rows1)
print(f"There are duplicate values which represents {percent_missing:.5f}% of the rows in our DataFrame")


In [ ]:
products_df.duplicated().value_counts(normalize=True)

In [ ]:
# Dropping all occurrances of duplicates except the first

products_df = products_df.drop_duplicates().copy()

In [ ]:
products_df.duplicated().sum() # this shows that there are no more duplicates in the data

### Look for Missing values


In [ ]:
# your code here
products_df.isnull().sum()

In [ ]:
products_df['type'].nunique() # we don"t have time to go through and check all of them so but filling them missing so we still have it for later
products_df['type'] = products_df['type'].fillna('missing')



In [ ]:
products_df['desc'] = products_df['desc'].fillna('missing')
products_df.isnull().sum()

In [ ]:
# we will dropna with price because we can not find the base price any other way.

products_df['price']=products_df['price'].dropna().copy()
products_df

### Check / Change Data types

In [ ]:
# your code here
orders_df.info()
orderlines_df.info()
products_df.info()
#price needs to be change to float, same with promo_price,

# Going to look at price and promo_price


See if I can easily change dtype to float. This might mean checking if weird characters are in the column.

In [ ]:
products_df["price"] = pd.to_numeric(products_df["price"]) # shows that there are multiple decimal points like in the order_lines price

Now I will see what the corrupted data looks like and how similar it is to a correct price information. I want to see if we can save the data.

In [ ]:
products_df[500:510]

In [ ]:
price_check_mask = orderlines_df['sku'] == 'DLK0050'

orderlines_df.loc[price_check_mask]

The promo price data is a mess in multiple ways if we want to keep to the timeline and have good data and results it is best to drop the promo price and use the day of sales price from orderlines to caluculate the discount.

price we have the same issue as the unit price and will need to drop the prices with the two decimal points. We will be recordings all loss of lines to be calculated later.

In [ ]:
# drop promo price
products_df = products_df.drop('promo_price', axis = 1).copy()
products_df



In [ ]:
products_df.info()


In [ ]:
(products_df['price'].str.count(r"\.")>1).sum() # 377 cells have the multipel decimal
(products_df['price'].str.count(r"\.")>1).sum()/products_df.shape[0]# 3% of the dataset is in this set

In [ ]:
# # remove multiple decimal points and transform price to intereger

products_df = products_df.loc[~(products_df['price'].str.count(r'\.')>1)]
products_df

In [ ]:
#transform price to intereger
#

products_df['price'] = pd.to_numeric(products_df['price'])
products_df.head()
products_df.info()
products_df.describe()# all values are postive to work with I would like to say that this data set is clean

# Write products df to csv.

products_df

In [ ]:
products_df.to_csv("products_clean.csv", index=False)

Below you will find other infomration I was interested in exploring.

In [ ]:
products_df.head()

In [ ]:
orderlines_df.loc[price_check_mask].agg({"unit_price":"mean"})

In [ ]:
oderlines_df.loc[]

In [ ]:
# Count the rows with more than one `.`
mult_decimal_rows = (products_df['price'].str.count(r"\.")>1).sum()

# Find the percentage of corrupted rows
percent_corrupted = (100 * mult_decimal_rows / products_df.shape[0])
print(f"{percent_corrupted:.2f}% of the rows in our DataFrame have multiple decimal points in the unit_price")

In [ ]:
url = "https://drive.google.com/file/d/1YRFxqSwXm9MCgB3y9hCMtATb_uO2Zm3s/view?usp=drive_link"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
brands = pd.read_csv(path)

In [ ]:
brands_df = brands.copy()

In [ ]:
brands_df.duplicated().sum() # no duplicates are found

In [ ]:
brands_df.isnull().sum() # no na found

In [ ]:
brands_df.head()

In [ ]:
brands_df.info()

In [ ]:
orders_df.tail()

In [ ]:
orderlines_df.tail()

In [ ]:
orders_df.groupby("state").count()

In [ ]:
orders_df.info()

In [ ]:


orders_df['state'].value_counts(normalize = 'True')

In [ ]:
type_1282_mask= products_df['type'] == '1282' # potenial name Apple Computer

products_df.loc[type_1282_mask, ['sku','name','desc','in_stock']].head(20)